In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2,threshold=50,linewidth=75)

In [2]:
bike_numpy = np.loadtxt('../data/p1ch4/bike-sharing-dataset/hour-fixed.csv',
                        dtype=np.float32,
                        delimiter=',',
                        skiprows=1,
                        converters={1:lambda x: float(x[8:10])})#skiprows =1 跳过第一行

In [3]:
bikes = torch.from_numpy(bike_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [4]:
bikes.shape,bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [5]:
daily_blikes = bikes.view(-1,24,bikes.shape[1])# .view(各个维度尺寸，，) -1表示自动计算该维度
daily_blikes.shape , daily_blikes.stride() , bikes.shape[1]

(torch.Size([730, 24, 17]), (408, 17, 1), 17)

In [6]:
daily_blikes = daily_blikes.transpose(1,2)#.transpose(1，2)将维度1和维度2 进行转置
daily_blikes.shape,daily_blikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [7]:
first_day = bikes[:24].long()
# bikes.shape,first_day.shape
weather_onehot = torch.zeros(first_day.shape[0],4)
first_day[:,9],first_day.shape

(tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
         2, 2]),
 torch.Size([24, 17]))

In [8]:
first_day[0], weather_onehot.shape,weather_onehot,bikes[:24].shape

(tensor([ 1,  1,  1,  0,  1,  0,  0,  6,  0,  1,  0,  0,  0,  0,  3, 13, 16]),
 torch.Size([24, 4]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         ...,
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]),
 torch.Size([24, 17]))

In [9]:
torch.cat((bikes[:24],weather_onehot),1)[:1]#将 bikes[:24],weather_onehot按 1 维度 进行拼接，这两个参数shape都是【24：**】，第一维度进行拼接，则是得到shape【24：(bikes.shape[1]+weather.shape[1])】

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          6.0000,  0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,
          3.0000, 13.0000, 16.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [10]:
torch.cat((bikes[:24],weather_onehot),1)[:1].shape

torch.Size([1, 21])

In [11]:
daily_weather_onehot = torch.zeros(daily_blikes.shape[0],4,
                                   daily_blikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [12]:
first_day[:,9].unsqueeze(1).long(),first_day[:,9].unsqueeze(1).long()-1

(tensor([[1],
         [1],
         [1],
         [1],
         [1],
         [2],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [1],
         [2],
         [2],
         [2],
         [2],
         [2],
         [3],
         [3],
         [2],
         [2],
         [2],
         [2]]),
 tensor([[0],
         [0],
         [0],
         [0],
         [0],
         [1],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [0],
         [1],
         [1],
         [1],
         [1],
         [1],
         [2],
         [2],
         [1],
         [1],
         [1],
         [1]]))

In [14]:
daily_weather_onehot.scatter_(1,daily_blikes[:,9,:].long().unsqueeze(1)-1,1.0)
daily_weather_onehot.shape, daily_blikes[:,9,:].shape,daily_blikes.shape,daily_blikes[:,9,:].long().unsqueeze(1).shape

(torch.Size([730, 4, 24]),
 torch.Size([730, 24]),
 torch.Size([730, 17, 24]),
 torch.Size([730, 1, 24]))

In [15]:
daily_blikes = torch.cat((daily_blikes,daily_weather_onehot),dim=1)
daily_blikes.shape

torch.Size([730, 21, 24])

In [16]:
daily_blikes[:,9,:] = (daily_blikes[:,9,:] - 1)/3.0

In [17]:
temp = daily_blikes[:,10,:]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_blikes[:,10,:] = ((daily_blikes[:,10,:]- temp_min)/(temp_max - temp_min))

In [18]:
temp = daily_blikes[:,10,:]
daily_blikes[:,10,:] = ((daily_blikes[:,10,:] - torch.mean(temp))/torch.std(temp))